### Install package

In [1]:
!pip install pytorch-adapt[ignite]

     |████████████████████████████████| 137 kB 12.8 MB/s 
     |████████████████████████████████| 107 kB 68.7 MB/s 
     |████████████████████████████████| 397 kB 65.6 MB/s 
     |████████████████████████████████| 254 kB 64.0 MB/s 


### Import packages

In [2]:
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.frameworks.ignite import Ignite
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.validators import IMValidator, ScoreHistory

### Create datasets and dataloaders

In [3]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".", download=True)
dc = DataloaderCreator(batch_size=32, num_workers=2)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/134178716 [00:00<?, ?it/s]

100%|██████████| 68007/68007 [00:11<00:00, 6001.09it/s]


### Create models, optimizers, hook, and validator

In [4]:
G = mnistG(pretrained=True)
C = mnistC(pretrained=True)
D = Discriminator(in_size=1200, h=256)
models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.0001}))

adapter = DANN(models=models, optimizers=optimizers)
validator = ScoreHistory(IMValidator())
trainer = Ignite(adapter, validator)

Downloading: "https://cornell.box.com/shared/static/tdx0ts24e273j7mf3r2ox7a12xh4fdfy" to /root/.cache/torch/hub/checkpoints/mnistG-68ee7945.pth


  0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading: "https://cornell.box.com/shared/static/j4zrogronmievq1csulrkai7zjm27gcq" to /root/.cache/torch/hub/checkpoints/mnistC-ac7b5a13.pth


  0%|          | 0.00/1.31M [00:00<?, ?B/s]

### Train and evaluate

In [5]:
best_score, best_epoch = trainer.run(datasets, dataloader_creator=dc, max_epochs=2)

[1/1843]   0%|          |it [00:00<?]

[1/1844]   0%|          |it [00:00<?]

[1/1843]   0%|          |it [00:00<?]

[1/1844]   0%|          |it [00:00<?]

In [6]:
print(f"best_score = {best_score}, best_epoch = {best_epoch}")

best_score = 1.2884315848350525, best_epoch = 2
